In [14]:
import json
with open('verb_candidates.json', 'r') as f:
    json_list = json.load(f)
    

### 1. Multiple Choice Question

In [32]:
import random

index_to_letter = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}

In [16]:
# options are verb phrases
question_bundle = {}
for sentence in json_list:
    group_id = sentence['gid']
    if group_id not in question_bundle:
        question_bundle[group_id] = [sentence]
    else:
        question_bundle[group_id].append(sentence)

In [48]:
questions = []
for group_id, sent_list in question_bundle.items():
    question = {}
    ppl_vp = {}
    tmp_vp_list = []
    id, gt_subj, gt_verb, gt_loc, dt_verb = None, None, None, None, None
    for sent in sent_list:
        if sent['tag'] == 'ground-truth':
            id = sent['gid']
            gt_subj = sent['subj']
            gt_verb = sent['vp']
            gt_loc = sent['loc']
        elif sent['tag'] == 'distractor':
            dt_verb = sent['vp']
        ppl_vp[sent['vp']] = sent['ppl']
        tmp_vp_list.append(sent['vp'])
    
    assert(any([id, gt_subj, gt_verb, gt_loc, dt_verb]) != None)
    question['id'] = id
    question['subj'] = gt_subj
    question['loc'] = gt_loc
    question['gt_vp'] = gt_verb
    question['dt_vp'] = dt_verb
    question['ppl'] = ppl_vp

    assert(len(tmp_vp_list) == 5)
    random.shuffle(tmp_vp_list) # we shuffle the option's list
    ans_idx = tmp_vp_list.index(gt_verb)
    
    q_text = f"What is/are {gt_subj} doing {gt_loc}?\nA. {tmp_vp_list[0]}\nB. {tmp_vp_list[1]}\nC. {tmp_vp_list[2]}\nD. {tmp_vp_list[3]}\nE. {tmp_vp_list[4]}"
    question['question'] = q_text
    question['answer'] = index_to_letter[ans_idx]
    question['image'] = f"Generate an image of {gt_subj} {gt_verb} {gt_loc} in realistic style"
    questions.append(question)

with open('verb_questions.json', 'w') as f:
    json.dump(questions, f)

In [49]:
with open('verb_questions.txt', 'w') as outfile:
    with open('verb_questions.json', 'r') as infile:
        j_list = json.load(infile)
        for question in j_list:
            outfile.write(f"Question{question['id']}:\n{question['image']}\n{question['question']}\n")
            outfile.write('\n')
            outfile.write(f"Answer: {question['answer']}\n")
            outfile.write(f"PPL: {question['ppl']}\n\n")
            outfile.write("="*10)
            outfile.write('\n')